In [1]:
import pandas as pd
import datetime as dt

pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

# The Gyro verification via sunshot azimuth

At fugro abourd m.v. Pioneer, I took part at a Sunshot-azimuth gyro calibration. noting down the times and made observations as well.

This notebook is a way to recreate the gyro calibration as per validation via the sunshot azimuth hour angle method. Using the Python astronomy library Skyfield to calculate the AZIMUTH of the sun by way of the hour angle method using the UTC time as inmput. 

## The original filesheet

![title](img/pioneer_sunshot_datasheet.png)

## Generated filesheet
The output.csv  imported below and, is generated in this notebook under "2.2. Multiple Azimuth method"

In [2]:
output = pd.read_csv('data/sunshots_output.csv', index_col='fix_nr')

# Deleted column Unnamed: 0 from output
output.drop(['Unnamed: 0'], axis=1, inplace=True)

mean_co = output['c-o'].mean()
std_co = output['c-o'].std()

print(output)
print('mean  c-o: {:.2f}'.format(mean_co))
print('stdef c-o: {:.2f}'.format(std_co))

               datetime_UTC  obs_sun_decimal  azimuth  obs_gyro_true  \
fix_nr                                                                 
1       2019-12-31 08:41:44       343.690000  138.873        155.063   
2       2019-12-31 08:43:16       344.100000  139.192        154.985   
3       2019-12-31 08:44:12       344.278889  139.387        155.055   
4       2019-12-31 08:44:52       344.368056  139.526        155.058   
5       2019-12-31 08:46:01       344.591667  139.766        155.076   
6       2019-12-31 08:46:42       344.755556  139.909        155.043   
7       2019-12-31 08:47:21       345.010556  140.045        154.940   
8       2019-12-31 08:48:12       345.451111  140.223        154.667   
9       2019-12-31 08:48:58       345.698611  140.383        154.588   
10      2019-12-31 09:07:08       349.303889  144.230        154.765   
11      2019-12-31 09:07:24       349.395556  144.287        154.764   
12      2019-12-31 09:07:45       349.459722  144.361        154

### Mean C-O & Stdef

In [3]:
print('mean  c-o: {:.2f}'.format(mean_co))
print('stdef c-o: {:.2f}'.format(std_co))

mean  c-o: 0.10
stdef c-o: 0.03


 ### Methodology Gyro verification via sunshot  
 The observation instument, be it a totalstation or a theodolite, is placed on the ships- centerline facing the bow, the angle of the centerline due to true north is used as baseline to compare with the angle of the gyro. The sun's observations relative to the baseline and the azimuth are used to reproduce the calculated gyroheading:
 
 In this case the calculated heading was computed by:  
 360 - OBS_sun + sun_azimuth
 
 The difference of the headings are thus:  
 The computed heading (c) - the observed heading (o)
 


 

![title](img/compass_obs.png)

## Raw import

In [4]:
raw = 'data/gyro_sunshot_31-12-19.csv'
dfr = pd.read_csv(raw, names=['fix_nr','time','obs_sun_D','obs_sun_M','obs_sun_S','d0','d1','d2','obs_gyro_true','d3','d4']) 

df = dfr.drop(['d0','d1','d2','d3','d4'], axis=1) # d0/d4 will be removed

df.head()

,fix_nr,time,obs_sun_D,obs_sun_M,obs_sun_S,obs_gyro_true
0,1,08:41:44,343,41,24,155.063
1,2,08:43:16,344,6,0,154.985
2,3,08:44:12,344,16,44,155.055
3,4,08:44:52,344,22,5,155.058
4,5,08:46:01,344,35,30,155.076


## DMS to decimal degrees with pandas using seperate columns.

In the dataframe above the columns from the observed sun are in the DMS format and are  3 components of one angle.

In the code below the DMS columns are merged into the decimal column.

In [5]:
# Create separate dataframe for DMS to decimal operations & datetime column
d1 = {'fix_nr':  dfr['fix_nr'], 
      'obs_sun_D': dfr['obs_sun_D'],
      'obs_sun_M': dfr['obs_sun_M'],
      'obs_sun_S': dfr['obs_sun_S']
    }
df1 = pd.DataFrame( data=d1 ) # dms 2 deg.deg

df1["obs_sun_decimal"] = df1[["obs_sun_D","obs_sun_M","obs_sun_S"]].apply(lambda row: row.values[0] + row.values[1]/60 + row.values[2]/3600, axis=1)
df1.head()

 
df['obs_sun_decimal'] = df1['obs_sun_decimal']
df

,fix_nr,time,obs_sun_D,obs_sun_M,obs_sun_S,obs_gyro_true,obs_sun_decimal
0,1,08:41:44,343,41,24,155.063,343.690000
1,2,08:43:16,344,6,0,154.985,344.100000
2,3,08:44:12,344,16,44,155.055,344.278889
3,4,08:44:52,344,22,5,155.058,344.368056
4,5,08:46:01,344,35,30,155.076,344.591667
...,...,...,...,...,...,...,...
15,16,09:08:50,349,42,35,154.768,349.709722
16,17,09:09:13,349,51,28,154.733,349.857778
17,18,09:09:33,350,3,0,154.597,350.050000
18,19,09:09:57,350,16,46,154.509,350.279444


## Datetime operations

In [6]:
# adding a date to the time in dfr
d2 = {'date': ['2019-12-31', '2019-12-31', '2019-12-31', '2019-12-31', '2019-12-31',
             '2019-12-31', '2019-12-31', '2019-12-31', '2019-12-31', '2019-12-31',
             '2019-12-31', '2019-12-31', '2019-12-31', '2019-12-31', '2019-12-31',
             '2019-12-31', '2019-12-31', '2019-12-31', '2019-12-31', '2019-12-31']
     }
df2= pd.DataFrame(data=d2)
df2['time'] = dfr.time # default time is UTC
df2['datetime_UTC'] = pd.to_datetime(df2['date'].astype(str) + ' ' + df2['time'].astype(str))

"""
DF TO DATETIME w/ Pandas:
https://stackoverflow.com/questions/53470304/convert-time-object-to-datetime-format-in-python-pandas

print(pd.to_datetime(df['date'].astype(str) + ' ' + df['time'].astype(str)))
"""
df2

# adding additional columns to df 
df['obs_sun_decimal'] = df1['obs_sun_decimal']
df['datetime_UTC']= df2['datetime_UTC']
col_order = ['fix_nr','datetime_UTC','obs_sun_D','obs_sun_M','obs_sun_S','obs_sun_decimal','obs_gyro_true'] 
df.drop(['time'], axis=1)
df = df.reindex(columns=col_order)
df

,fix_nr,datetime_UTC,obs_sun_D,obs_sun_M,obs_sun_S,obs_sun_decimal,obs_gyro_true
0,1,2019-12-31 08:41:44,343,41,24,343.690000,155.063
1,2,2019-12-31 08:43:16,344,6,0,344.100000,154.985
2,3,2019-12-31 08:44:12,344,16,44,344.278889,155.055
3,4,2019-12-31 08:44:52,344,22,5,344.368056,155.058
4,5,2019-12-31 08:46:01,344,35,30,344.591667,155.076
...,...,...,...,...,...,...,...
15,16,2019-12-31 09:08:50,349,42,35,349.709722,154.768
16,17,2019-12-31 09:09:13,349,51,28,349.857778,154.733
17,18,2019-12-31 09:09:33,350,3,0,350.050000,154.597
18,19,2019-12-31 09:09:57,350,16,46,350.279444,154.509


In [7]:
 # cdt: converted datetime 
df2['cdt'] = df['datetime_UTC'].dt.strftime('%Y,%m,%d,%H,%M,%S')
print(df2)
cdt = df2.cdt

          date       time        datetime_UTC                  cdt
0   2019-12-31   08:41:44 2019-12-31 08:41:44  2019,12,31,08,41,44
1   2019-12-31   08:43:16 2019-12-31 08:43:16  2019,12,31,08,43,16
2   2019-12-31   08:44:12 2019-12-31 08:44:12  2019,12,31,08,44,12
3   2019-12-31   08:44:52 2019-12-31 08:44:52  2019,12,31,08,44,52
4   2019-12-31   08:46:01 2019-12-31 08:46:01  2019,12,31,08,46,01
5   2019-12-31   08:46:42 2019-12-31 08:46:42  2019,12,31,08,46,42
6   2019-12-31   08:47:21 2019-12-31 08:47:21  2019,12,31,08,47,21
7   2019-12-31   08:48:12 2019-12-31 08:48:12  2019,12,31,08,48,12
8   2019-12-31   08:48:58 2019-12-31 08:48:58  2019,12,31,08,48,58
9   2019-12-31   09:07:08 2019-12-31 09:07:08  2019,12,31,09,07,08
10  2019-12-31   09:07:24 2019-12-31 09:07:24  2019,12,31,09,07,24
11  2019-12-31   09:07:45 2019-12-31 09:07:45  2019,12,31,09,07,45
12  2019-12-31   09:08:03 2019-12-31 09:08:03  2019,12,31,09,08,03
13  2019-12-31   09:08:19 2019-12-31 09:08:19  2019,12,31,09,0

# Skyfield - Astronomical library 

https://rhodesmill.org/skyfield/

The website states:
"Skyfield computes positions for the stars, planets, and satellites in orbit around the Earth. Its results should agree with the positions generated by the United States Naval Observatory and their Astronomical Almanac to within 0.0005 arcseconds (half a “mas” or milliarcsecond)." 

### Specify location / time of fix  

Note that both the location and the time in utc can easely be changed, with the skyfield library an azimmut can easely be genarated based these two imputs on request.

In [10]:
# location Port of Den Helder, Nieuwe diep:
lat = 52+(57/60)+(26.9/3600)
lon =  4+(46/60)+(37.5/3600)
height_m = 6
# fix1 @ 2019-12-31 08:41:44 UTC >>> 2019,12,31,08,41,44

## Single angle sunshot method

In [11]:
# src: https://rhodesmill.org/skyfield/positions.html#azimuth-and-altitude-from-a-geographic-position
# Sunshot Azimuth - hour angle method
from skyfield.api import N,S,E,W, wgs84
from skyfield.api import load
import pandas as pd

ts = load.timescale()
t = ts.utc(2019, 12, 31)
planets = load('de421.bsp')
earth, sun = planets['earth'], planets['sun']

# Altitude and azimuth in the sky for a specific geographic location
earth = planets['earth']
Nieuwe_diep = earth + wgs84.latlon(lat * N, lon * E, elevation_m=height_m)
astro = Nieuwe_diep.at(ts.utc(2019, 12, 31, 8, 41, 44)).observe(sun) # fix1 @ 2019-12-31 08:41:44 UTC

app = astro.apparent()

alt, az, distance = app.altaz()
#print('alt: ' + alt.dstr())
print('az: ' + az.dstr())
#print(distance)


#print('lat, lon: ' + str(lat), str(lon))
#dt_utc = df2['datetime_UTC']

print('az: {:.3f}'.format(az.degrees)) # desired output for azimuth in decimal degrees
print('az: '+ az.dstr(format=u'{0}{1}°{2:02}′{3:02}.{4:0{5}}″'))

az: 138deg 52' 22.3"
az: 138.873
az: 138°52′22.3″


## Multiple Azimuth method

In [13]:
# copying the data from df2 and pasting it between ''' here ''' for string comprehension.
# for now this is the quick & dirty way
import io
data = '''           date       time        datetime_UTC                  cdt
0   2019-12-31   08:41:44 2019-12-31 08:41:44  2019,12,31,08,41,44
1   2019-12-31   08:43:16 2019-12-31 08:43:16  2019,12,31,08,43,16
2   2019-12-31   08:44:12 2019-12-31 08:44:12  2019,12,31,08,44,12
3   2019-12-31   08:44:52 2019-12-31 08:44:52  2019,12,31,08,44,52
4   2019-12-31   08:46:01 2019-12-31 08:46:01  2019,12,31,08,46,01
5   2019-12-31   08:46:42 2019-12-31 08:46:42  2019,12,31,08,46,42
6   2019-12-31   08:47:21 2019-12-31 08:47:21  2019,12,31,08,47,21
7   2019-12-31   08:48:12 2019-12-31 08:48:12  2019,12,31,08,48,12
8   2019-12-31   08:48:58 2019-12-31 08:48:58  2019,12,31,08,48,58
9   2019-12-31   09:07:08 2019-12-31 09:07:08  2019,12,31,09,07,08
10  2019-12-31   09:07:24 2019-12-31 09:07:24  2019,12,31,09,07,24
11  2019-12-31   09:07:45 2019-12-31 09:07:45  2019,12,31,09,07,45
12  2019-12-31   09:08:03 2019-12-31 09:08:03  2019,12,31,09,08,03
13  2019-12-31   09:08:19 2019-12-31 09:08:19  2019,12,31,09,08,19
14  2019-12-31   09:08:34 2019-12-31 09:08:34  2019,12,31,09,08,34
15  2019-12-31   09:08:50 2019-12-31 09:08:50  2019,12,31,09,08,50
16  2019-12-31   09:09:13 2019-12-31 09:09:13  2019,12,31,09,09,13
17  2019-12-31   09:09:33 2019-12-31 09:09:33  2019,12,31,09,09,33
18  2019-12-31   09:09:57 2019-12-31 09:09:57  2019,12,31,09,09,57
19  2019-12-31   09:10:20 2019-12-31 09:10:20  2019,12,31,09,10,20
'''
df2 = pd.read_csv(io.StringIO(data), sep=' \s+', engine='python')
df2['datetime_UTC'] = pd.to_datetime(df2['datetime_UTC'])

df2['cdt'] = df2['datetime_UTC'].dt.strftime('%Y,%m,%d,%H,%M,%S')
# note I changed the formatting to remove spaces for later parsing

def calc_az(tutc):
    yr=int(tutc.split(',')[0])
    mo=int(tutc.split(',')[1])
    da=int(tutc.split(',')[2])
    hr=int(tutc.split(',')[3])
    mi=int(tutc.split(',')[4])
    se=int(tutc.split(',')[5])

    ts = load.timescale()
    t = ts.utc(2019, 12, 31)
    planets = load('de421.bsp')
    earth, sun = planets['earth'], planets['sun']

    # Altitude and azimuth in the sky for a specific geographic location
    earth = planets['earth']
    Nieuwe_diep = earth + wgs84.latlon(lat * N, lon * E, elevation_m=height_m)
    # astro = Nieuwe_diep.at(ts.utc(2019, 12, 31, 8, 41, 44)).observe(sun)
    astro = Nieuwe_diep.at(ts.utc(yr, mo, da, hr, mi, se)).observe(sun)

    app = astro.apparent()

    alt, az, distance = app.altaz()
    # print('alt: ' + alt.dstr())
    # print('az:  ' + az.dstr())
    # print(distance)

    # print('lat, lon: ' + str(lat), str(lon))
    dt_utc = df2['datetime_UTC']

    print('{:.3f}'.format(az.degrees)) # desired output for azimuth in decimal degrees
    # print('az: '+ az.dstr(format=u'{0}{1}°{2:02}′{3:02}.{4:0{5}}″'))
    # print('\n'*2)
    return

In [12]:
df2['cdt'].apply(calc_az) # copy  Azimuth angles from print statment below
df2['azimuth'] = [
138.873,
139.192,
139.387,
139.526,
139.766,
139.909,
140.045,
140.223,
140.383,
144.230,
144.287,
144.361,
144.426,
144.483,
144.536,
144.593,
144.676,
144.747,
144.833,
144.915
]

138.873
139.192
139.387
139.526
139.766
139.909
140.045
140.223
140.383
144.230
144.287
144.361
144.426
144.483
144.536
144.593
144.676
144.747
144.833
144.915


In [31]:
#print(df)
output = df
output['azimuth'] = df2.azimuth
output = output.drop(['obs_sun_D','obs_sun_M','obs_sun_S'], axis=1)
output['calc_gyro_true'] = (360 -output.obs_sun_decimal) + output.azimuth
output['c-o'] = output.calc_gyro_true - output.obs_gyro_true

# reorder of columns
column_reorder = ["fix_nr", "datetime_UTC", "obs_sun_decimal", "azimuth", "obs_gyro_true", "calc_gyro_true", "c-o"]
output=output.reindex(columns=column_reorder)
output.to_csv("data/sunshots_output.csv")
output

,fix_nr,datetime_UTC,obs_sun_decimal,azimuth,obs_gyro_true,calc_gyro_true,c-o
0,1,2019-12-31 08:41:44,343.690000,138.873,155.063,155.183000,0.120000
1,2,2019-12-31 08:43:16,344.100000,139.192,154.985,155.092000,0.107000
2,3,2019-12-31 08:44:12,344.278889,139.387,155.055,155.108111,0.053111
3,4,2019-12-31 08:44:52,344.368056,139.526,155.058,155.157944,0.099944
4,5,2019-12-31 08:46:01,344.591667,139.766,155.076,155.174333,0.098333
...,...,...,...,...,...,...,...
15,16,2019-12-31 09:08:50,349.709722,144.593,154.768,154.883278,0.115278
16,17,2019-12-31 09:09:13,349.857778,144.676,154.733,154.818222,0.085222
17,18,2019-12-31 09:09:33,350.050000,144.747,154.597,154.697000,0.100000
18,19,2019-12-31 09:09:57,350.279444,144.833,154.509,154.553556,0.044556
